In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import time
import numpy as np


In [ ]:
import os


In [ ]:
%matplotlib ipympl

import matplotlib.pyplot as plt
#plt.rcParams['figure.figsize'] = (9.5, 7.5)



In [ ]:
from pfs.lam.opdb import *
from pfs.lam.fileHandling import *
from pfs.lam.detFocusAnalysis import *
from pfs.lam.detAnalysis import *
from pfs.lam.linePeaksList import filterPeakList

In [ ]:
import lsst.daf.persistence as dafPersist

In [ ]:
experimentId = 125 # 411µ
rerun = "defocus"
#rerun = "dcb2"

cam = "b2"

basePath = '/drp/analysis/sm2/'

peaklist = "../peakList/B2_on_SM2_peakList.csv"


In [ ]:
experimentId = 719 # 411µ
rerun = "ginga"
#rerun = "dcb2"

cam = "r2"

basePath = '/drp/analysis/sm3/'

peaklist = "../peakList/R2_on_SM3_peakList.csv"


In [ ]:
np.arange(523,526+1,1)

In [ ]:

expIds = np.append(133,np.arange(134,136+1,1))
expIds = np.append(expIds,np.arange(145,145+1,1))
expIds = np.append(expIds,np.arange(152,155+1,1))
expIds = np.append(expIds,np.arange(158,158+1,1))

In [ ]:
expIds = np.arange(731,731+1,1)

In [ ]:
expIds = np.append(157,160)

In [ ]:
expIds = np.append(expIds,np.arange(646,655+1,1))


In [ ]:
expIds

In [ ]:
doRoiPlot = True

In [ ]:


drpPath = "/data/drp"
repo = "sps"


arm = cam[0]
specId = int(cam[1])
dataId = dict(arm=arm, spectrograph=specId)
repoRoot = f"{drpPath}/{repo}"
print(repoRoot)
print(os.path.join(repoRoot,  "rerun", rerun, "detrend"))
print(os.path.join(repoRoot, "CALIB"))

butler = dafPersist.Butler( os.path.join(repoRoot, "rerun", rerun, "detrend"), \
                           calibRoot=os.path.join(repoRoot, "CALIB"))
#butler = dafPersist.Butler( os.path.join(repoRoot, "rerun", rerun), \
#                           calibRoot=os.path.join(repoRoot, "CALIB"))
butler.getKeys('raw')


# define defaut parameters
roi_size = 24
seek_size = 60
com = True  # Center Of Massfabrice.madec@free.fr
doBck = True
head = 0
tail = 0
criteria = 'EE5'
doPrint = False


for experimentId in expIds:
    
    visitStart, visitEnd = getVisitRange(experimentId)

    print(visitStart, visitEnd)

    csvPath = os.path.join(basePath,f"Exp{experimentId}",rerun,f"roi{roi_size}",f"doBck{doBck}")
    print(csvPath)

    print(f'Start visit {visitStart} of {cam} with {peaklist}')
    print(f"{drpPath}/{repo}/rerun/{rerun}/detrend/")

    if doRoiPlot:
        # get lamp used to filter the list of peaks
        dataId.update(visit=int(visitStart))
        lamps = butler.queryMetadata('raw', ['lamps'], dataId)
        print(f"lamps used: {lamps}")


        peaks = filterPeakList(peaklist, arm, lamps)

        waves = peaks.wavelength.unique()
        print(waves)

        # filter wave if needed
        peaks = peaks[peaks.wavelength != waves[-1]]

        exp = butler.get("calexp", visit=int(visitStart), arm=cam[0])
        plotRoiPeak(exp.image.array, peaks, raw=True, roi_size=60, )

    for v in range(visitStart, visitEnd+1):
        dataId.update(visit=int(v))
        df = ImageQualityToCsv(butler, dataId, peaks, csv_path=csvPath,\
                               com=com, doBck=doBck, EE=[3,5], seek_size=seek_size,\
                               doFit=False, doLSF=False,  doSep=True, \
                               mask_size=20, threshold= 50, subpix = 5 , maxPeakDist=80,\
                               maxPeakFlux=40000, minPeakFlux=2000,\
                               doPlot=False, doPrint=doPrint)
